In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from ktvgl_functions import init_parameters, fit_multivariate_t, L_update, w_update,  u_update, a_update, V_update, dual_update, residual_Update, update_parameters, updateADMMpenalties, L_star,L_from_w,D_star,D_from_w

In [ ]:
log_rw_returns = pd.read_csv("log_rw_returns.csv")
winLen = 200
Xraw = log_rw_returns[0:winLen]

maxIter = 200
update_rho = True
update_eta = True
early_stopping = False


X = scale(Xraw.to_numpy())

# number of nodes
p = X.shape[1] # n_features
k = 1 # n_clusters

# number of observations
T_n = X.shape[0]      # n_observations in Frame
sigma_e = np.exp(10) # Student T Distribution Standard Dev. 
alpha = 2/(T_n*sigma_e)
beta = 2*np.log(sigma_e)/T_n

print("alpha",alpha)
rho = 2    # ADMM hyperparameter. 
gamma = 10 # hyperparameter that controls the sparsity of VAR coefficients
eta = 1e-8 # hyperparameter that controls the regularization to obtain a k-component graph
#tau = 2 # Not used
#mu = 2 # Not used


# BOOK KEEPING
# residual vectors
primal_lap_residual = np.array([], dtype=float)
primal_deg_residual = np.array([], dtype=float)
dual_residual       = np.array([], dtype=float)

lagrangian = np.array([], dtype=float)
rho_seq = np.array([], dtype=float)
eta_seq = np.array([], dtype=float)

elapsed_time = np.array([], dtype=float)


mu, Sigma, nu, history = fit_multivariate_t( X, nu_init=30.0, max_iter=200, tol=1e-6, fix_nu=None, jitter=1e-6, verbose=False)
w, Aw, a, w_lagged, L_n, Phi_n, u_n, mu_vec, d, z_n, V = init_parameters(X,k)
nu = 10.02571
#print("w",w)
#print("Aw",Aw)
#print("a",a)
#print("w_lagged",w_lagged)
#print("L_n",L_n)
#print("Phi_n",Phi_n)
#print("u_n",u_n)
#print("mu_vec",mu_vec)
#print("d",d)
#print("z_n",z_n)
#print("V",V)

#print("nu",nu)




for iter in range(0,maxIter):


    

    #wUpdate = w_update(X, w, w_lagged,a,  nu, L_n, L_nUpdate,  u_n, mu_vec, z_n, Phi_n, V, rho, d, eta, beta)
    wUpdate = w_update(X, w, w_lagged, a, nu, L_n, u_n, mu_vec, z_n, Phi_n, V, rho, d, eta, beta)            
    #print("wUpdate",wUpdate)
    u_nUpdate = u_update(wUpdate,a,w_lagged,mu_vec,rho,alpha)
    #print("u_nUpdate",u_nUpdate)
    # x_update()
    aUpdate =a_update( w_lagged, wUpdate, u_nUpdate, mu_vec, gamma, rho)
    #print("aUpdate",aUpdate)
    vUpdate = V_update(wUpdate,k)
    #print("vUpdate",vUpdate)

    L_nUpdate = L_update(wUpdate, Phi_n, rho, k)
    #print("L_nUpdate")
    #print(L_nUpdate)

    Phi_nUpdate, mu_vecUpdate, z_nUpdate, Phi_n_res, u_n_res, z_n_res = dual_update(rho, wUpdate, L_nUpdate, Phi_n, aUpdate, w_lagged, mu_vec, u_nUpdate, d, z_n)
    #print("Phi_nUpdate")
    #print(Phi_nUpdate)
    #print("mu_vecUpdate",mu_vecUpdate)
    #print("z_nUpdate",z_nUpdate)
    primal_lap_residual, primal_deg_residual, dual_residual = residual_Update(rho, L_n, L_nUpdate, Phi_n_res, z_n_res, primal_lap_residual, primal_deg_residual, dual_residual)
    #print("primal_lap_residual",primal_lap_residual) 
    #print("primal_deg_residual",primal_deg_residual) 
    #print("dual_residual",dual_residual)
    
    rho, eta, rho_seq, eta_seq, has_converged = updateADMMpenalties( update_rho, rho, rho_seq,L_n, k, wUpdate, update_eta, eta, eta_seq, early_stopping)
    #print("rho",rho) 
    #print("eta",eta) 
    #print("rho_seq",rho_seq)
    #print("eta_seq",eta_seq)
    #print("has_converged",has_converged)

    
    L_n, w, u_n, a, V, Phi_n, mu_vec, z_n = update_parameters( L_nUpdate, wUpdate, u_nUpdate, aUpdate, vUpdate, Phi_nUpdate, mu_vecUpdate, z_nUpdate)

    if has_converged:
        break





alpha 4.539992976248485e-07


In [3]:
L_n

array([[ 1.0000001 , -0.2122269 , -0.29774266, -0.20305625, -0.2869743 ],
       [-0.2122269 ,  1.00000047, -0.23225074, -0.30945238, -0.24607046],
       [-0.29774266, -0.23225074,  0.99999993, -0.24527122, -0.22473532],
       [-0.20305625, -0.30945238, -0.24527122,  0.9999998 , -0.24221995],
       [-0.2869743 , -0.24607046, -0.22473532, -0.24221995,  1.00000002]])

In [ ]:

def dual_update( rho, wUpdate, L_nUpdate, Phi_n, aUpdate, w_lagged, mu_vec, u_nUpdate, d, z_n):

    '''
    rho:      (1)                              ADMM Hyperparameter  

    wUpdate:   ( n_features*(n_features-1)/2 )  Updated Graph Weights for current Frame
    L_nUpdate: ( n_features, n_features )       Updated Laplacian Constrained Graph Matrix
    Phi_n:     ( n_features, n_features )       Laplacian Constrained Graph Matrix Dual

    aUpdate:   ( n_features*(n_features-1)/2 )  Updated VAR Weights for Previous Frame
    w_lagged:  ( n_features*(n_features-1)/2 )  Graph Weights for previous Frame  
    u_nUpdate: ( n_features*(n_features-1)/2 )  Updated Temporal Consistency Parameter
    mu_vec:    ( n_features*(n_features-1)/2 )  Temporal Consistency Parameter Dual 

    z_n:         (n_features)                   Degree Dual 
    d:           (1)                            Degree Constraint
    
    '''

    # update Phi
    Phi_n_res =  L_from_w(wUpdate) - L_nUpdate
    Phi_nUpdate = Phi_n + rho * Phi_n_res

    # update mu
    u_n_res = u_nUpdate - wUpdate + np.multiply(aUpdate,w_lagged) 
    mu_vecUpdate = mu_vec + rho * u_n_res

    # update z
    z_n_res = D_from_w(wUpdate) - d
    z_nUpdate = z_n + rho * z_n_res

    return Phi_nUpdate, mu_vecUpdate, z_nUpdate, Phi_n_res, u_n_res, z_n_res

Phi_nUpdate, mu_vecUpdate, z_nUpdate, Phi_n_res, u_n_res, z_n_res = dual_update(rho, wUpdate, L_nUpdate, Phi_n, aUpdate, w_lagged, mu_vec, u_nUpdate, d, z_n)
print("Phi_nUpdate")
print(Phi_nUpdate)
print("mu_vecUpdate",mu_vecUpdate)
print("z_nUpdate",z_nUpdate)

In [ ]:
def softThresh(v,thr):

    '''
    v:   ( )  Value(s) to be Threshed  
    thr: (1) Threshold Value
    '''

    temp = np.abs(v) - thr
    temp = temp*(temp>0)
    return(np.sign(v)*temp)


def u_update(wUpdate,a,w_lagged,mu_vec,rho,alpha):

    '''
    wUpdate:  ( n_features*(n_features-1)/2 )  Updated Graph Weights for current Frame
    a:        ( n_features*(n_features-1)/2 )  VAR Weights for Previous Frame
    w_lagged: ( n_features*(n_features-1)/2 )  Graph Weights for previous Frame

    mu_vec:   ( n_features*(n_features-1)/2 )  Temporal Consistency Parameter Dual
    alpha:    (1)                              Temporal Weight Sparsity Hyperparameter
    rho:      (1)                              ADMM Hyperparameter
    '''

    # Update u
    u_nTemp  = wUpdate - a*w_lagged - mu_vec/rho
    print("u_nTemp", u_nTemp)
    thr = alpha/(rho)
    print("thr", thr)
    u_nUpdate = softThresh(u_nTemp, thr)
    return u_nUpdate

print("wUpdate",wUpdate)
u_nUpdate = u_update(wUpdate,a,w_lagged,mu_vec,rho,alpha)
print("u_nUpdate",u_nUpdate)

In [ ]:
def softThresh(v,thr):

    '''
    v:   ( )  Value(s) to be Threshed  
    thr: (1) Threshold Value
    '''

    temp = abs(v) - thr
    temp = temp*(temp>0)
    return(np.sign(v)*temp)

def a_update2( w_lagged, wUpdate, u_nUpdate, mu_vec, gamma, rho):
    '''
    wUpdate:  ( n_features*(n_features-1)/2 )  Updated Graph Weights for current Frame
    u_nUpdate:( n_features*(n_features-1)/2 )  Updated Temporal Consistency Parameter
    mu_vec:   ( n_features*(n_features-1)/2 )  Temporal Consistency Parameter Dual 
    gamma:    (1)                              Hyperparameter that controls the sparsity of VAR coefficients
    rho:      (1)                              ADMM Hyperparameter    
    '''
    
    aUpdate = np.zeros_like(w_lagged) # ( n_features*(n_features-1)/2 )  Updated VAR Weights for current Frame
    
    f_temp = wUpdate - u_nUpdate - mu_vec/rho
    f_temp[f_temp<0] = 0
    idx = w_lagged > 0
    thr = gamma/(rho* w_lagged[idx]**2)
    aUpdate[idx] = softThresh(f_temp[idx]/w_lagged[idx], thr)
    aUpdate [~idx] = 0
    return aUpdate

aUpdate = a_update2( w_lagged, wUpdate, u_nUpdate, mu_vec, gamma, rho)

aUpdate

In [ ]:
def w_update2(X, w, w_lagged, a, nu, L_n, u_n, mu_vec, z_n, Phi_n, V, rho, d, eta, beta):
    """
    NumPy translation of the R weight update (student-t case), matching the paper's Eq. (20)–(21).

    Shapes:
      X: (T_n, p)
      w, w_lagged, a, u_n, mu_vec: (m,) with m = p*(p-1)//2
      L_n, L_nUpdate, Phi_n: (p, p)
      z_n: (p,)
      V: (p, k)
      d, nu, rho, eta, beta: scalars (or d can be length-p)

    Requires user-provided operators:
      L_star: (p,p) -> (m,)
      D_star: (p,)  -> (m,)
      D_from_w: (m,) -> (p,)
    """

    T_n, p = X.shape
    m = w.shape[0]

    Lw = L_from_w(w)
    LstarLw = L_star(Lw)
    DstarDw = D_star(np.diag(Lw))

    LstarSq = [None] * T_n
    for t in range(T_n):
        x = X[t, :]
        LstarSq[t] = L_star(np.outer(x, x))  # L_star: (p,p) -> (m,)
    #print("LstarSq",LstarSq)

    
    LstarSweighted = np.repeat(0, .5*p*(p-1))
    S_tilde = np.repeat(0, .5*p*(p-1))
    #print("Init LstarSweighted",LstarSweighted)
    for t in range(T_n):
        #LstarSweighted = LstarSweighted + LstarSq[t] * (p + nu) / (float(np.dot(w, LstarSq[t])) + nu )
        x = X[t, :]
        #LstarSweighted = LstarSweighted + L_star(np.outer(x, x)) * ( (p + nu) / ( L_star(x.T * Lw * x)  + nu ) )
        S_tilde = S_tilde + L_star(np.outer(x, x)) * ( (p + nu) / ( L_star(x.T * Lw * x)  + nu ) )

    #print("LstarSweighted/T_n",S_tilde/T_n)

    #a_w = LstarSweighted/T_n + L_star(eta * (V @ V.T) + Phi_n - rho * (L_n)) + rho*LstarLw
    a_w = S_tilde/T_n + L_star(eta * (V @ V.T) + Phi_n + rho * (Lw -L_n))
    #print("a_w",a_w)


    d_vec = np.full(p, d, dtype=float) if np.ndim(d) == 0 else d
    b_w = -mu_vec - rho * (u_n + a * w_lagged) + D_star(z_n - rho * (d_vec - D_from_w(w)))
    #print("b_w",b_w)

    grad = a_w + b_w
    #print("aw + bw", grad)

    ratio = 1 / (rho*(4*p-1))
    w_new = (1-rho*ratio)*w - ratio *  grad

    thr = np.sqrt( 2*beta *ratio )
    w_new[w_new< thr] = 0

    return w_new

In [ ]:
wUpdate  = w_update2(X, w, w_lagged, a, nu, L_n, L_nUpdate, u_n, mu_vec, z_n, Phi_n, V, rho, d, eta, beta)
wUpdate 

In [ ]:
Lw = L_from_w(w)
x = X[1, :]
x.T * Lw * x

In [ ]:
def w_update3(X, w, w_lagged, a, nu, L_n, u_n, mu_vec, z_n, Phi_n, V, rho, d, eta, beta):
    '''
    X:           (n_samples, n_features)        Data Array For Window (Frame) Length

    w:           (n_features*(n_features-1)/2)  Graph Weights for current Frame
    w_lagged:    (n_features*(n_features-1)/2)  Graph Weights for previous Frame
    a            (n_features*(n_features-1)/2)  VAR Weights for Previous Frame
    u_n:         (n_features*(n_features-1)/2)  Temporal Consistency Parameter
    mu_vec:      (n_features*(n_features-1)/2)  Temporal Consistency Parameter Dual

    L_n:         (n_features, n_features)       Updated Laplacian Constrained Graph Matrix
    Phi_n:       (n_features, n_features)       Laplacian Constrained Graph Matrix Dual
    
    
    z_n:         (n_features)                   Degree Dual 
    d:           (1)                            Degree Constraint
    

    V:           (n_features, n_clusters)       Penalty to control rank of Ln

    nu           (1)                            Student T Degrees Of Freedom Parameter
    rho:         (1)                            ADMM Hyperparameter    
    eta          (1)                            Hyperparameter that controls the regularization to obtain a k-component graph
    beta         (1)                            Weight Sparsity Hyperparameter
    '''

    T_n = X.shape[0] # Window (Frame) Length
    p = X.shape[1]   # Number Of Features

    Lw = L_from_w(w) # (n_features, n_features) Laplacian with current weights

    S_tilde = np.repeat(0, .5*p*(p-1))

    for t in range(T_n):
        x = X[t, :]
        S_tilde = S_tilde + L_star(np.outer(x, x)) * ( (p + nu) / ( L_star(x.T * Lw * x)  + nu ) )


    a_w = S_tilde/T_n + L_star(eta * (V @ V.T) + Phi_n + rho * (Lw -L_n))

    d_vec = np.full(p, d, dtype=float) if np.ndim(d) == 0 else d
    b_w = -mu_vec - rho * (u_n + a * w_lagged) + D_star(z_n - rho * (d_vec - D_from_w(w)))
 

    ratio = 1 / (rho*(4*p-1))
    c_w = (1-rho*ratio)*w - ratio *  (a_w + b_w)

    thr = np.sqrt( 2*beta *ratio )
    w_new = np.multiply(c_w > thr, c_w)

    return w_new


In [ ]:
wUpdate  = w_update3(X, w, w_lagged, a, nu, L_n, u_n, mu_vec, z_n, Phi_n, V, rho, d, eta, beta)
wUpdate 